In [1]:
!nvidia-smi

Thu Nov  9 07:37:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   27C    P8    14W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# pip install ultralytics

In [3]:
# pip install roboflow

In [4]:
# pip install --upgrade ipython

In [5]:
pip install ultralytics==8.0.73

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [7]:
!wandb login

/bin/bash: line 1: wandb: command not found


In [8]:
# import wandb
# import random

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

# # simulate training
# epochs = 10
# offset = random.random() / 5
# for epoch in range(2, epochs):
#     acc = 1 - 2 ** -epoch - random.random() / epoch - offset
#     loss = 2 ** -epoch + random.random() / epoch + offset
    
#     # log metrics to wandb
#     wandb.log({"acc": acc, "loss": loss})
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

In [9]:
from IPython import display
from IPython.display import display, Image
from ultralytics import YOLO

# display.clear_output()

# YOLO 객체 생성
# yolo = YOLO()

# verify 메서드 실행
# yolo.check()

In [10]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [11]:
# Pip install method (recommended)

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.73 🚀 Python-3.10.9 torch-2.0.1 CUDA:0 (Tesla T4, 14972MiB)
Setup complete ✅ (8 CPUs, 30.9 GB RAM, 66.7/96.7 GB disk)


In [12]:
from ultralytics import YOLO
from IPython.display import display, Image

In [1]:
"""
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("trafficlightdetect").project("traffic-light-ke5b5")
dataset = project.version(10).download("yolov8")
"""

'\nfrom roboflow import Roboflow\nrf = Roboflow(api_key="")\nproject = rf.workspace("trafficlightdetect").project("traffic-light-ke5b5")\ndataset = project.version(10).download("yolov8")\n'

In [2]:
"""
↓ Pre-Trained Model download

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace().project("traffic-light-ke5b5")
model = project.version(8).model
"""

'\n↓ Pre-Trained Model download\n\nfrom roboflow import Roboflow\nrf = Roboflow(api_key="")\nproject = rf.workspace().project("traffic-light-ke5b5")\nmodel = project.version(8).model\n'

In [15]:
import yaml
import torch
from torchvision import transforms
from PIL import Image as PILImage
from tqdm import tqdm
from PIL import Image

In [16]:
# # 원래 이미지가 RGBA 형식인지 RGB 형식인지 판별하는 코드
# original_image_path = '/content/Traffic_light/train/images/-3-_jpg.rf.04a86cd5626655ea5bb84f86189cea46.jpg'  # 원래 이미지 파일 경로 설정
# original_image = Image.open(original_image_path)
# image_mode = original_image.mode  # 이미지의 모드를 가져옵니다. 'RGBA' 또는 'RGB'일 수 있습니다.
# print(f"원래 이미지의 모드: {image_mode}")

In [17]:
# 데이터 증강을 위한 변환 정의
transform = transforms.Compose([
    transforms.RandomResizedCrop(640),  # 이미지 크기를 무작위로 자르고 640x640 크기로 조절합니다.
    transforms.RandomHorizontalFlip(),  # 50% 확률로 좌우 반전을 수행합니다.
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 색상, 대비, 채도 및 색조를 무작위로 조절합니다.
    transforms.RandomRotation(10),  # 최대 10도까지 무작위로 회전시킵니다.
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # 무작위로 이동, 크기 조절, 기울임 변환을 수행합니다.
    transforms.GaussianBlur(kernel_size=3),  # 가우시안 블러를 적용합니다.
    transforms.ToTensor(),  # 이미지를 텐서로 변환합니다.
])

In [18]:
# 원본 이미지 경로 및 증강된 이미지 저장 경로 설정
original_data_dir = '/content/Traffic_light/train/images'
augmented_data_dir = '/content/Traffic_light/augmented_images'

# 기존 'data.yaml' 파일의 경로
data_yaml_path = '/content/Traffic_light/data.yaml'

In [20]:
# 기존 'data.yaml' 파일을 로드합니다.
with open(data_yaml_path, 'r') as file:
    data_yaml = yaml.safe_load(file)

# 'train' 섹션에 이미지 경로를 추가합니다.
data_yaml['train'] = []  # 'train' 섹션을 빈 리스트로 초기화합니다.

for image_file in tqdm(os.listdir(original_data_dir), desc='Augmenting Images'):
    original_image_path = os.path.join(original_data_dir, image_file)
    original_image = Image.open(original_image_path).convert("RGB")
    
    # 6가지의 데이터 증강 방법을 적용하여 이미지를 6번 증강합니다.
    for i in range(6):
        augmented_image = transform(original_image)
        augmented_image_path = os.path.join(augmented_data_dir, f"{os.path.splitext(image_file)[0]}_{i}.jpg")
        augmented_image_pil = transforms.ToPILImage()(augmented_image)
        augmented_image_pil.save(augmented_image_path)
        
        # 업데이트된 이미지 경로를 'train' 섹션에 추가합니다.
        data_yaml['train'].append(os.path.relpath(augmented_image_path, '/content/Traffic_light'))

# 업데이트된 'data.yaml' 파일을 저장합니다.
with open(data_yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

print(f"Augmented data saved to {augmented_data_dir}")
print(f"Updated paths in {data_yaml_path}")

Augmenting Images: 100%|██████████| 701/701 [06:13<00:00,  1.88it/s]


Augmented data saved to /content/Traffic_light/augmented_images
Updated paths in /content/Traffic_light/data.yaml


In [ ]:
# # data.yaml에서 'train' 키의 값을 리스트로 초기화
# data_yaml['train'] = []

# # 데이터 증강 및 저장
# for image_file in tqdm(image_files, desc='데이터 증강 중', unit='이미지'):
#     # 원본 이미지 불러오기
#     original_image_path = os.path.join(original_data_dir, image_file)
#     original_image = Image.open(original_image_path).convert("RGB")
    
#     # 변환 적용
#     augmented_image = transform(original_image)
    
#     # 저장할 경로 설정
#     augmented_image_filename = f"augmented_{image_file}"
#     augmented_image_path = os.path.join(augmented_data_dir, augmented_image_filename)
    
#     # 이미지 저장
#     transforms.ToPILImage()(augmented_image).save(augmented_image_path)
    
#     # data.yaml 파일 업데이트 (증강된 이미지 경로 추가)
#     data_yaml['train'].append(os.path.relpath(augmented_image_path, '/content/Traffic_light'))

# # 업데이트된 data.yaml 파일 저장
# with open(data_yaml_path, 'w') as file:
#     yaml.dump(data_yaml, file, default_flow_style=False)

# print(f"Updated data.yaml with augmented images.")

In [21]:
# data.yaml 파일의 경로를 업데이트합니다.
new_paths = {
    'test': '/content/Traffic_light/test/images',  # 테스트 이미지의 저장 경로를 설정합니다.
    'train': '/content/Traffic_light/train/images',  # 훈련 이미지의 저장 경로를 설정합니다.
    'val': '/content/Traffic_light/valid/images'  # 검증 이미지의 저장 경로를 설정합니다.
}

# data.yaml 파일을 읽어옵니다.
with open(data_yaml_path, 'r') as file:
    data_yaml = yaml.safe_load(file)

# 이미지 저장 경로를 업데이트합니다.
data_yaml['test'] = new_paths['test']
data_yaml['train'] = new_paths['train']
data_yaml['val'] = new_paths['val']

# 업데이트된 data.yaml 파일을 저장합니다.
with open(data_yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

print(f"Updated paths in {data_yaml_path}")  # 업데이트된 data.yaml 파일의 경로를 출력합니다.

Updated paths in /content/Traffic_light/data.yaml


In [22]:
# # Define the new paths relative to the location of data.yaml
# new_paths = {
#     'test': '/content/Traffic_light/test/images',
#     'train': '/content/Traffic_light/train/images',
#     'val': '/content/Traffic_light/valid/images'
# }

# # Path to your data.yaml file
# data_yaml_path = '/content/Traffic_light/data.yaml'

# # Read in the existing data.yaml file
# with open(data_yaml_path, 'r') as file:
#     data_yaml = yaml.safe_load(file)

# # Update the paths
# data_yaml['test'] = new_paths['test']
# data_yaml['train'] = new_paths['train']
# data_yaml['val'] = new_paths['val']

# # Write the updated data.yaml back to the file
# with open(data_yaml_path, 'w') as file:
#     yaml.dump(data_yaml, file, default_flow_style=False)

# print(f"Updated paths in {data_yaml_path}")

In [23]:
stop

NameError: name 'stop' is not defined

In [24]:
import wandb

In [26]:
# WandB 초기화
wandb.init(project='awesome_project_try2')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fcd70391150>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fcd72eebb20, raw_cell="# WandB 초기화
wandb.init(project='awesome_project_tr.." store_history=True silent=False shell_futures=True cell_id=None>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

In [27]:
lrf = 0.001  # final layer의 학습률 조정
batch = 32  # 배치 크기 조정
epochs = 200  # 에폭 수 조정

In [28]:
# WandB에 학습 파라미터 기록
wandb.config.lr = lrf
wandb.config.batch_size = batch
wandb.config.epochs = epochs

In [29]:
pip install wandb==0.15.3

Note: you may need to restart the kernel to use updated packages.


In [30]:
from ultralytics.yolo.engine.model import YOLO

In [31]:
from wandb.integration.yolov8 import add_callbacks as add_wandb_callbacks

In [32]:
model = YOLO('yolov8m.pt')  # 미리 학습된 모델 로드 (학습에 권장됨)
add_wandb_callbacks(model, project='awesome_project_try2')

results = model.train(data='/content/Traffic_light/data.yaml', epochs=epochs, imgsz=640, batch=batch, lrf=lrf)

wandb: WARNING The wandb callback is currently in beta and is subject to change based on updates to `ultralytics yolov8`.
wandb: WARNING         The callback is tested and supported for ultralytics v8.0.43 and above.
wandb: WARNING         Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.
wandb: WARNING         
New https://pypi.org/project/ultralytics/8.0.208 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.73 🚀 Python-3.10.9 torch-2.0.1 CUDA:0 (Tesla T4, 14972MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/Traffic_light/data.yaml, epochs=200, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      6/200      12.6G      1.899      1.381      1.034        147        640: 100%|██████████| 22/22 [00:19<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.20it/s]
                   all        199        495      0.667      0.425      0.463        0.2

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      7/200      12.6G      1.929      1.306      1.044         98        640: 100%|██████████| 22/22 [00:19<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.20it/s]
                   all        199        495      0.513      0.428      0.424      0.185

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      8/200      12.6G      1.893      1.295      1.082    

     25/200      12.6G      1.618     0.9145     0.9726        104        640: 100%|██████████| 22/22 [00:19<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.22it/s]
                   all        199        495       0.72      0.527      0.575      0.282

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     26/200      12.6G       1.49     0.8546     0.9516        122        640: 100%|██████████| 22/22 [00:19<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.19it/s]
                   all        199        495      0.703      0.567      0.576       0.28

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     27/200      12.6G      1.571     0.8665      0.955        108        640: 100%|██████████| 22/22 [00:19<00:00,  1.12it/s]
           

     63/200      12.6G      1.226     0.6258     0.8704        108        640: 100%|██████████| 22/22 [00:19<00:00,  1.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.19it/s]
                   all        199        495      0.816      0.573      0.661      0.367

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     64/200      12.6G      1.215      0.643     0.8769        104        640: 100%|██████████| 22/22 [00:19<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.21it/s]
                   all        199        495       0.79       0.63      0.682      0.361

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     65/200      12.5G      1.243     0.6609     0.8907        152        640: 100%|██████████| 22/22 [00:19<00:00,  1.13it/s]
           

    101/200      12.6G      1.124     0.5812     0.8592        108        640: 100%|██████████| 22/22 [00:19<00:00,  1.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.15it/s]
                   all        199        495       0.78       0.63      0.694      0.393

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    102/200      12.5G       1.02     0.5225     0.8412        162        640: 100%|██████████| 22/22 [00:19<00:00,  1.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.19it/s]
                   all        199        495      0.804      0.616      0.678      0.379

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    103/200      12.6G     0.9835     0.5133     0.8438        121        640: 100%|██████████| 22/22 [00:19<00:00,  1.13it/s]
           

    139/200      12.6G     0.8541      0.439     0.8189        129        640: 100%|██████████| 22/22 [00:19<00:00,  1.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.46it/s]
                   all        199        495      0.797      0.667       0.72      0.416
Stopping training early as no improvement observed in last 50 epochs. Best results observed at epoch 89, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

139 epochs completed in 0.967 hours.
Optimizer stripped from runs/detect/train21/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train21/weights/best.pt, 52.0MB

Validating runs/detect/train21/weights/best.pt...
Ultralytics YOLOv8.0.73 🚀 Python-3.10.9 torch-2.0.1 CUDA:0 (Tesla T4, 14972MiB)
Model summary (fused): 218 layers, 25842076 parameters, 0 gradients, 78.7 GFLOP

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr/pg0,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▂▇███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▂▇███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▅▄▅▅▅▇▆▆▇▇▇▇▇▇▇▇▇▇█▇█▇█████████████████
metrics/mAP50-95(B),▁▄▃▄▄▄▅▅▅▆▆▆▆▇▆▆▆▇▇▇▇▇▇▇▇▇█▇██▇█▇▇█▇█▇██
metrics/precision(B),▁▆▄▆▆▅▇▆▆▆▇▇█▇▇▆▇▆▇▇▇▇▇▇█▇▇▇█▇▇▇▇█▇▇▇█▇▇
metrics/recall(B),▁▄▄▅▅▅▆▆▆▇▇▇▆█▇█▇█▇█▇█▇█▇███▇▇███▇▇▇▇▇█▇
train/box_loss,▇██▇▇▆▆▅▆▅▅▅▅▄▅▅▄▄▄▄▄▃▃▃▃▂▂▂▃▂▂▂▂▂▁▁▁▁▁▁
train/cls_loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/dfl_loss,▆▇█▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂
